In [69]:
import pandas as pd
from datetime import datetime
import numpy as np
# import scrapy
# from scrapy.crawler import CrawlerProcess

In [70]:
# class CareerhubSpider(scrapy.Spider):
#     name = 'letterboxd'
#     allowed_domains = ['letterboxd.com']
#     start_urls = ['https://letterboxd.com/iainm/films/diary/']

#     def parse(self, response):
#         links = response.css('#diary-table > tbody > tr > td.td-film-details > h3 > a::attr(href)').extract()
#         ratings = response.css('#diary-table > tbody > tr > td.td-rating.rating-green > div > span::attr(class)').extract()
#         rating_dates = response.css('#diary-table > tbody > tr > td.td-day.diary-day.center > a::attr(href)').extract()

#         for link, rating, rating_date in zip(links, ratings, rating_dates):
#             link = response.urljoin(link)
#             yield scrapy.Request(url=link, callback=self.parse_review, meta={'rating': rating, 'date_rated': rating_date})

#         next_page = response.css(
#             'a.next::attr(href)').extract_first()
#         if next_page:
#             next_page = response.urljoin(next_page)
#             yield scrapy.Request(next_page, callback=self.parse)

#     def parse_review(self, response):
#         title_links = response.css('#content > div > div > section > section > section > h2 > span.film-title-wrapper > a::attr(href)').extract()
        
#         for title_link in title_links:
#             title_link = response.urljoin(title_link)
#             yield scrapy.Request(url=title_link, callback=self.parse_details, meta={'rating': response.meta['rating'], 'date_rated': response.meta['date_rated']})

#     def parse_details(self, response):
#         yield {
#             'title': response.css('#featured-film-header > h1::text').extract_first(),
#             'year': response.css('#featured-film-header > p > small > a::text').extract_first(),
#             'rating': response.meta['rating'],
#             'date_rated': response.meta['date_rated'],
#             'duration': response.css('#film-page-wrapper > div.col-17 > section.section.col-10.col-main > p::text').extract_first().strip(),
#             'director': response.xpath('//a[contains(@class, "text-slug") and contains(@href, "director")]/text()').extract(),
#             'country': response.xpath('//a[contains(@class, "text-slug") and contains(@href, "films/country")]/text()').extract(),
#             'genre': response.xpath('//a[contains(@class, "text-slug") and contains(@href, "/films/genre")]/text()').extract(),
#             'actors': response.css('#tab-cast > div > p > a::text').extract(),
#             'image': response.css('#js-poster-col img::attr(src)').extract_first(),
#             'backdrop': response.css("#backdrop::attr(data-backdrop)").extract_first(),
#             'summary': response.css('div.truncate > p::text').extract_first()
#         }

In [71]:
df = pd.read_csv("letterboxd.csv")
df.head()

,title,year,rating,date_rated,duration,director,country,genre,actors,image,backdrop,summary
0,The Cremator,1969,rating rated-7,/iainm/films/diary/for/2020/07/05/,100 mins \n\t\t\t\t\n\t\t\t\t\tMore details at,Juraj Herz,NaN,"thriller,horror,drama","Jiří Menzel,Rudolf Hrušínský,Vlasta Chramostov...",https://a.ltrbxd.com/resized/film-poster/4/0/4...,https://a.ltrbxd.com/resized/sm/upload/1s/fo/5...,Kopfrkingl enjoys his job at a crematorium in ...
1,Hero,2002,rating rated-7,/iainm/films/diary/for/2020/05/30/,99 mins \n\t\t\t\t\n\t\t\t\t\tMore details at,Zhang Yimou,China,"drama,history,adventure,action","Jet Li,Tony Leung Chiu-Wai,Maggie Cheung,Donni...",https://a.ltrbxd.com/resized/sm/upload/ox/qe/4...,https://a.ltrbxd.com/resized/sm/upload/fs/r4/p...,One man defeated three assassins who sought to...
2,The Emigrants,1971,rating rated-8,/iainm/films/diary/for/2020/05/30/,191 mins \n\t\t\t\t\n\t\t\t\t\tMore details at,Jan Troell,Sweden,"drama,history","Max von Sydow,Liv Ullmann,Allan Edwall,Eddie A...",https://a.ltrbxd.com/resized/film-poster/2/4/2...,https://a.ltrbxd.com/resized/sm/upload/t5/8s/c...,"A Swedish peasant family, ravaged by poverty, ..."
3,Faces Places,2017,rating rated-7,/iainm/films/diary/for/2020/05/26/,89 mins \n\t\t\t\t\n\t\t\t\t\tMore details at,"Agnès Varda,JR",France,documentary,"Agnès Varda,JR",https://a.ltrbxd.com/resized/film-poster/3/8/4...,https://a.ltrbxd.com/resized/sm/upload/ps/37/l...,Director Agnès Varda and photographer/muralist...
4,The Decline of Western Civilization Part III,1998,rating rated-8,/iainm/films/diary/for/2020/06/05/,86 mins \n\t\t\t\t\n\t\t\t\t\tMore details at,Penelope Spheeris,USA,"music,documentary","Rick Wilder,Flea,Stephen Chambers,Gary Fredo",https://a.ltrbxd.com/resized/film-poster/2/0/3...,https://a.ltrbxd.com/resized/sm/upload/g2/xh/0...,The Decline of Western Civilization III is a 1...


In [72]:
df["rating"] = df["rating"].str.split("-", expand=True)[1]

In [73]:
df["date_rated"] = df["date_rated"].str[-11:].str[0:10]

In [74]:
df["duration"] = df["duration"].str.split(expand=True)[0]

In [75]:
df["genre"] = df["genre"].str.title()

In [76]:
df["genre"] = df["genre"].str.replace(",", ", ")

In [77]:
df["director"] = df["director"].str.replace(",", ", ")

In [78]:
df['date_rated'] = pd.to_datetime(df.date_rated, format='%Y/%m/%d')

In [79]:
df["decade"] = df["year"].astype(str).str[:3] + "0s"

In [80]:
df[["rating", "duration"]] = df[["rating", "duration"]].astype(int)

In [81]:
#df = df[df["director"].notnull()]
df["director"] = df["director"].fillna("Director not specified")

In [82]:
df["actors"] = df["actors"].fillna("Cast not specified")

In [83]:
df["genre"] = df["genre"].fillna("Genre not specified")

In [84]:
df.set_index("title", inplace=True)

In [85]:
df.loc[["Teddy Bomb"], ["country"]] = "Canada"
df.loc[["Daisies", "Loves of a Blonde", "The Shop on Main Street"], ["country"]] = "Czechia"
df.loc[["Three Businessmen"], ["country"]] = "USA,Japan,Netherlands,UK"
df.loc[["That Evening Sun"], ["country"]] = "USA"
df.loc[["All Tomorrow's Parties"], ["country"]] = "UK"
df.loc[["Stalker"], ["country"]] = "Russian Federation"
df.loc[["The Cremator"], ["country"]] = "Czechia"

In [86]:
blank_country = df.country.isnull()

In [87]:
df[blank_country]

,year,rating,date_rated,duration,director,country,genre,actors,image,backdrop,summary,decade
title,,,,,,,,,,,,


In [88]:
df.reset_index(inplace=True)

In [90]:
df.to_csv("../all_letterboxd.csv")

# Directors

In [54]:
directors = df["director"].str.split(",", expand=True)

In [55]:
directors = directors.unstack().reset_index(name="director")

In [56]:
directors.drop(columns=["level_0", "level_1"], inplace=True)

In [57]:
directors["director"] = directors["director"].str.strip()

In [58]:
directors.dropna(inplace=True)

In [59]:
directors["count"] = 1

In [60]:
directors = pd.DataFrame(directors.groupby("director")["count"].sum())

In [61]:
directors.sort_values(by="count", ascending=False, inplace=True)

In [62]:
directors = directors[:20]

In [63]:
directors.to_csv("../directors.csv")

In [91]:
dir_bar = df.iloc[:,[0, 2, 5]]
dir_bar.head()

,title,rating,director
0,The Cremator,7,Juraj Herz
1,Hero,7,Zhang Yimou
2,The Emigrants,8,Jan Troell
3,Faces Places,7,"Agnès Varda, JR"
4,The Decline of Western Civilization Part III,8,Penelope Spheeris


In [92]:
dir_bar = dir_bar.assign(director=dir_bar["director"].str.split(",")).explode("director")

In [93]:
dir_bar["director"] = dir_bar["director"].str.strip()

In [94]:
pivot = pd.pivot_table(dir_bar, index=["director"], columns=["title"], margins=True, aggfunc=[np.mean, len])

In [95]:
pivot = pivot.stack("title")

In [96]:
pivot = pivot.reset_index()

In [97]:
pivot = pivot.loc[pivot.director != "All"]

In [98]:
pivot.columns = pivot.columns.droplevel(1)

In [99]:
pivot

,director,title,len,mean
0,Abbas Kiarostami,All,4.0,8.0
1,Abbas Kiarostami,Certified Copy,1.0,8.0
2,Abbas Kiarostami,Close-Up,1.0,9.0
3,Abbas Kiarostami,Taste of Cherry,1.0,7.0
4,Abbas Kiarostami,The Wind Will Carry Us,1.0,8.0
...,...,...,...,...
1783,Ágnes Hranitzky,Werckmeister Harmonies,1.0,7.0
1784,Éric Rohmer,All,3.0,8.0
1785,Éric Rohmer,My Night at Maud's,1.0,8.0
1786,Éric Rohmer,Pauline at the Beach,1.0,8.0


In [100]:
pivot.to_csv("../test.csv", index=False)

In [101]:
dir_bar.to_csv("../dir_bar.csv", index=False)

# Decade

In [102]:
decade = df.groupby("decade").agg({"title": "size", "rating": "mean"}).rename(columns={"title": "count", "rating": "avg_rating"})
decade

,count,avg_rating
decade,,
1910s,4,7.250000
1920s,23,7.565217
1930s,27,7.333333
1940s,43,7.488372
1950s,87,7.701149
1960s,130,7.607692
1970s,161,7.571429
1980s,141,7.340426
1990s,128,7.429688


In [103]:
decade.to_csv("../decade_breakdown.csv")

# Release Year

In [104]:
release_year = df.iloc[:, [0, 1]]

In [105]:
min_year = release_year["year"].min()

In [106]:
max_year = release_year["year"].max()

In [107]:
year_range = np.arange(min_year, max_year + 1)

In [108]:
release_year_group = release_year.groupby(["year"]).count()

In [109]:
release_year_group = release_year_group.reindex(year_range).fillna(0)

In [110]:
release_year_group.to_csv("../release_year.csv")

# Watch date

In [111]:
w_date = df.set_index("date_rated")

In [112]:
w_date = w_date.groupby("date_rated").count()

In [113]:
w_date = pd.DataFrame(w_date.iloc[:, 1])

In [114]:
first_w_date = pd.Timestamp("2015-04-25")
last_w_date = w_date.index.max()

In [115]:
w_date_range = pd.date_range(start=first_w_date, end=last_w_date)

In [116]:
w_date = w_date.reindex(w_date_range).fillna(0)

In [117]:
monthly = w_date.groupby(pd.Grouper(freq="MS")).sum()

In [118]:
monthly.rename(columns={"year": "count"}, inplace=True)

In [119]:
monthly.to_csv("../watch_date.csv", index_label="date")

# Genres

In [120]:
genres = df.iloc[:, [0, 7]]

In [121]:
genres = genres.assign(genre=genres["genre"].str.split(", ")).explode("genre")

In [122]:
genres.drop(columns="title", inplace=True)

In [123]:
genres = genres.drop_duplicates().dropna()

In [124]:
genres = genres[genres["genre"] != "Tv Movie"]

In [125]:
genres = genres[genres["genre"] != "Genre not specified"]

In [126]:
genres = genres.sort_values("genre")

In [127]:
genres.to_csv("../genre_list.csv", index=False)

# Countries

In [128]:
countries = df.iloc[:, [0, 6]]

In [129]:
countries = countries.assign(country=countries["country"].str.split(",")).explode("country")

In [130]:
countries.drop(columns="title", inplace=True)

In [131]:
countries = countries.drop_duplicates().dropna()

In [132]:
countries.sort_values(by="country", inplace=True)

In [133]:
countries.to_csv("../countries.csv", index=False)